# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

### Описание данных

•	id — уникальный идентификатор скважины;

•	f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);

•	product — объём запасов в скважине (тыс. баррелей).


### Задача

Построить модель машинного обучения, которая поможет определить регион, добыча в котором принесет компании наибольшую прибыль.

Цель - выбрать лучший регион для скважин.

## Загрузка и подготовка данных

In [1]:
# Импортируем все необходимое для работы
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Отключаем предупреждения
import warnings
warnings.filterwarnings('ignore')

# Откроем файлы 
try:
    df_0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
    df_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
    df_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')
except:
    df_0 = pd.read_csv('geo_data_0.csv')
    df_1 = pd.read_csv('geo_data_1.csv')
    df_2 = pd.read_csv('geo_data_2.csv')
    

In [2]:
# Выведем первые 10 строк
df_0.head(10)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


In [3]:
# Выведем первые 10 строк
df_1.head(10)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


In [4]:
# Выведем первые 10 строк
df_2.head(10)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


In [5]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
print(df_0.isna().sum())
print()
print(df_1.isna().sum())
print()
print(df_2.isna().sum())

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


In [9]:
print(df_0.duplicated().sum())
print()
print(df_1.duplicated().sum())
print()
print(df_2.duplicated().sum())

0

0

0


In [10]:
print(df_0.describe())
print()
print(df_1.describe())
print()
print(df_2.describe())

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%      

In [11]:
df_0 = df_0.drop('id', axis=1)
df_1 = df_1.drop('id', axis=1)
df_2 = df_2.drop('id', axis=1)

In [12]:
print(df_0[['f0', 'f1', 'f2']].corr())

          f0        f1        f2
f0  1.000000 -0.440723 -0.003153
f1 -0.440723  1.000000  0.001724
f2 -0.003153  0.001724  1.000000


In [13]:
print(df_1[['f0', 'f1', 'f2']].corr())

          f0        f1        f2
f0  1.000000  0.182287 -0.001777
f1  0.182287  1.000000 -0.002595
f2 -0.001777 -0.002595  1.000000


In [14]:
print(df_2[['f0', 'f1', 'f2']].corr())

          f0        f1        f2
f0  1.000000  0.000528 -0.000448
f1  0.000528  1.000000  0.000779
f2 -0.000448  0.000779  1.000000


Вывод.

Данные готовы к работе. Пропуски и дубликаты отсутствуют. Корреляции между признаками не обнаружено.

Также мы удалили столбец ['id'], для обучения модели этот признак неинформативен.

## Обучение и проверка модели

Разделим данные на обучающую и валидационную выборки с использованием функции StandardScaler для масштабирования признаков

In [15]:
RANDOM_STATE = np.random.RandomState(12345)

In [16]:
def split_scale_df(df, target_ss = 'product', test_size_ss = 0.25, random_state = RANDOM_STATE):
    
    # Разделим данные на features - признаки и target - целевой признак
    features = df.drop(target_ss, axis = 1)
    target = df[target_ss]
    
    # Поделим данные на обучающую выборку и валидационную
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size = test_size_ss, random_state = RANDOM_STATE)
    
    # Масштабируем признаки
    numeric = ['f0', 'f1', 'f2']
    scaler = StandardScaler()
    features_train[numeric] = scaler.fit_transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    
    return features_train[numeric], features_valid[numeric], target_train, target_valid

In [17]:
# применяем функцию
features_train_0, features_valid_0, target_train_0, target_valid_0 = split_scale_df(df_0)
features_train_1, features_valid_1, target_train_1, target_valid_1 = split_scale_df(df_1)
features_train_2, features_valid_2, target_train_2, target_valid_2 = split_scale_df(df_2)

In [18]:
print(features_train_0.shape)

(75000, 3)


In [19]:
print(target_train_0.shape)

(75000,)


In [20]:
print(features_valid_1.shape)

(25000, 3)


In [21]:
print(target_valid_1.shape)

(25000,)


Напишем функцию для обучения моделей линейной ирегрессии и сделаем предсказания по валидачионной выборке

In [22]:
def lr_model(features_train, features_valid, target_train, target_valid):
    
    # Запускаем обучение
    model_lr = LinearRegression()
    model_lr.fit(features_train, target_train)
    
    predicted_valid = model_lr.predict(features_valid)
    
    # считаем RMSE для модели
    rmse = mean_squared_error(target_valid, predicted_valid, squared = False)
    
    # считаем средний запас
    target_mean = target_valid.mean()
    
    return predicted_valid, rmse, model_lr, target_mean


In [23]:
predicted_valid_0, rmse_0, model_lr_0, target_mean_0 = lr_model(
    features_train_0, features_valid_0, target_train_0, target_valid_0)

predicted_valid_1, rmse_1, model_lr_1, target_mean_1 = lr_model(
    features_train_1, features_valid_1, target_train_1, target_valid_1)

predicted_valid_2, rmse_2, model_lr_2, target_mean_2 = lr_model(
    features_train_2, features_valid_2, target_train_2, target_valid_2)

In [24]:
print('Регион 1: Средний запас предсказанного сырья =', predicted_valid_0.mean())
print()
print('RMSE =', rmse_0)
print()
print('Истинный средний запас сырья =', target_mean_0)

Регион 1: Средний запас предсказанного сырья = 92.59256778438035

RMSE = 37.5794217150813

Истинный средний запас сырья = 92.07859674082927


In [25]:
print('Регион 2: Средний запас предсказанного сырья =', predicted_valid_1.mean())
print()
print('RMSE =', rmse_1)
print()
print('Истинный средний запас сырья =', target_mean_1)

Регион 2: Средний запас предсказанного сырья = 68.76995145799754

RMSE = 0.889736773768065

Истинный средний запас сырья = 68.77162424984647


In [26]:
print('Регион 3: Средний запас предсказанного сырья =', predicted_valid_2.mean())
print()
print('RMSE =', rmse_2)
print()
print('Истинный средний запас сырья =', target_mean_2)

Регион 3: Средний запас предсказанного сырья = 95.087528122523

RMSE = 39.958042459521614

Истинный средний запас сырья = 94.74895871720241


#### Вывод
Все моделли показывают хорошие результаты, близкие к истинным.

Лучше всего модель работает на втором регионе, показатель RMSE = 0.88. 

В третьем регионе модель показывает самый высокий уровень предсказаний, но её RMSE = 39. Это означает, что модель дает недостаточно точные результаты и опираться на них мы не можем.

## Подготовка к расчёту прибыли

In [27]:
# бюджет на разработку
budget = 10_000_000_000  

barrel_price = 450_000  

# нефтяные скважины для разработки
oil_well = 200 

# все нефтяные скважины
oil_well_all = 500 

In [28]:
# Рассчитаем безубыточный объем сырья для новой скважины
favorable_volume = budget / (barrel_price * oil_well)

print('Безубыточный объем сырья для новой скважины:', favorable_volume)

Безубыточный объем сырья для новой скважины: 111.11111111111111


In [29]:
print('Регион 1')
print()
print('Истинный средний объем =', target_mean_0)
print()
print('Отличие от безубыточного объема =', target_mean_0 - favorable_volume)

Регион 1

Истинный средний объем = 92.07859674082927

Отличие от безубыточного объема = -19.032514370281845


In [30]:
print('Регион 2')
print()
print('Истинный средний объем =', target_mean_1)
print()
print('Отличие от безубыточного объема =', target_mean_1 - favorable_volume)

Регион 2

Истинный средний объем = 68.77162424984647

Отличие от безубыточного объема = -42.339486861264646


In [31]:
print('Регион 3')
print()
print('Истинный средний объем =', target_mean_2)
print()
print('Отличие от безубыточного объема =', target_mean_2 - favorable_volume)

Регион 3

Истинный средний объем = 94.74895871720241

Отличие от безубыточного объема = -16.3621523939087


### Вывод.

В каждом регионе средний объем сырья оказался меньше неоходимого для безубыточной добычи. Во время разработки скважин всегда есть высокие риски неокупаемости средств компании, затраченных на построение новых скважин. Возможно следует обратить внимание на другие скважины, чтобы подыскать более надежные варианты.

## Расчёт прибыли и рисков 

In [32]:
def profit(predicted_valid, target_valid):
    predicted = pd.Series(predicted_valid).sort_values(ascending = False)
    target = target_valid.iloc[predicted.index][:oil_well]
    profit = target.sum() * barrel_price - budget
    
    return profit

In [33]:
R1_profit = profit(predicted_valid_0, target_valid_0.reset_index(drop = True))
print('Регион 1')
print('Прибыль =', R1_profit)

Регион 1
Прибыль = 3320826043.1398506


In [34]:
R2_profit = profit(predicted_valid_1, target_valid_1.reset_index(drop = True))
print('Регион 2')
print('Прибыль =', R2_profit)

Регион 2
Прибыль = 2415086696.681511


In [35]:
R3_profit = profit(predicted_valid_2, target_valid_2.reset_index(drop = True))
print('Регион 3')
print('Прибыль =', R3_profit)

Регион 3
Прибыль = 2539915945.842947


Воспользуемся методом Bootstrap для более точного анализа потенциальной прибыли и рисков.

In [36]:
def bootstrap(predicted_valid, target_valid):
    values = []

    for i in range(1000):
        s_target = target_valid.reset_index(drop=True).sample(
            n = oil_well_all, replace = True, random_state = RANDOM_STATE)
        s_predicted = predicted_valid[s_target.index]

        # Рассчитаем прибыль
        profit_s = profit(s_predicted, s_target)
        values.append(profit_s)

    # Рассчитаем среднюю прибыль, 95% доверительный интервал и риск убытков
    values = pd.Series(values)
    risk_loss = (values < 0).mean()
    profit_mean = values.mean()
    confidence_interval = (values.quantile(0.025), values.quantile(0.975))
    

    return risk_loss, confidence_interval, profit_mean

In [37]:
region_risk_0, region_confidence_0, region_profit_mean_0 = bootstrap(
    predicted_valid_0, target_valid_0)
print('Регион 1:')
print('Риск убытков: {:.1%}'.format(region_risk_0))
print('Средняя прибыль:', region_profit_mean_0)
print('Доверительный интервал:', region_confidence_0)

Регион 1:
Риск убытков: 6.1%
Средняя прибыль: 394284411.3405507
Доверительный интервал: (-69450566.85672115, 915496165.8311511)


In [38]:
region_risk_1, region_confidence_1, region_profit_mean_1 = bootstrap(
    predicted_valid_1, target_valid_1)
print('Регион 2:')
print('Риск убытков: {:.1%}'.format(region_risk_1))
print('Средняя прибыль:', region_profit_mean_1)
print('Доверительный интервал:', region_confidence_1)

Регион 2:
Риск убытков: 0.7%
Средняя прибыль: 454736359.0973578
Доверительный интервал: (61071811.297698975, 855914149.1428356)


In [39]:
region_risk_2, region_confidence_2, region_profit_mean_2 = bootstrap(
    predicted_valid_2, target_valid_2)
print('Регион 3:')
print('Риск убытков: {:.1%}'.format(region_risk_2))
print('Средняя прибыль:', region_profit_mean_2)
print('Доверительный интервал:', region_confidence_2)

Регион 3:
Риск убытков: 7.6%
Средняя прибыль: 353664097.5248346
Доверительный интервал: (-162650946.9422927, 847761844.5147682)


### Вывод.

Наилучшим вариантом будет 2-й регион, вероятност убытков в нем составила 0.7%. В этом регионе наблюдается лучший показатель средней прибыли.

Цель достигнута. Два этих параметра полностью удоблетворяют условия поставленной задачи, уровень риска меньше 2,5%.

# Общий вывод

1. Мы подготовили данные к работе. Проверили на отсутствие пропусков и дубликатов. Корреляции между признаками оказалась незначительной.Также мы удалили столбец ['id'], для обучения моделей этот признак неинформативен.

2. При проверке моделей все моделли показали хорошие результаты, близкие к истинным.

Лучше всего модель работает на втором регионе.

Средний запас предсказанного сырья = 68.76995145799754

RMSE = 0.889736773768065

Истинный средний запас сырья = 68.77162424986196

3. В процессе подготовки к расчету прибыли мы выяснили, что в каждом регионе средний объем сырья оказался меньше неоходимого для безубыточной добычи. Во время разработки скважин всегда есть высокие риски неокупаемости средств компании, затраченных на построение новых скважин. 

4. Написали функцию для расчета прибыли. С помощью метода Bootstrap получили более точный анализ потенциальной прибыли и рисков.

5. Наилучшим вариантом будет 2-й регион. Так же этом регионе наблюдается лучший показатель средней прибыли.

Риск убытков: 0.7%

Средняя прибыль: 454736359.09735775